In [ ]:
%pip install fastapi 
%pip install influxdb_client
%pip install minio

In [54]:
from base import LocalGateway, base_logger, PeriodicTrigger, BaseEventFabric, ExampleEventFabric

from influxdb_client import InfluxDBClient
from fastapi import FastAPI, Request
from datetime import datetime, timedelta
from minio import Minio
from io import BytesIO
import pandas as pd

# -------------------------- CONFIGURATION --------------------------

# InfluxDB credentials and database details
INFLUX_TOKEN = "http://192.168.1.132:8086"
INFLUX_ORG = "wise2024"
INFLUX_USER = "admin"
INFLUX_PASS = "secure_influx_iot_user"

VIZ_COMPONENT_URL = "http://192.168.1.132:9000"
SIF_SCHEDULER = ("SCH_SERVICE_NAME", "192.168.1.132:30032")

# Minio credentials and database details
MINIO_ENDPOINT = "192.168.1.132:9090"
MINIO_ACCESS_KEY = "peUyeVUBhKS7DvpFZgJu"
MINIO_SECRET_KEY = "J5VLWMfzNXBnhrm1kKHmO7DRbnU5XzqUO1iKWJfi"
MINIO_BUCKET = "models"

# Influx Buckets
BUCKETS = ["1_2_2", "1_2_7", "1_3_10", "1_3_11", "1_3_14", "1_4_12", "1_4_13"]  
BUCKETS_PIR = ["1_2_2", "1_3_11", "1_3_14", "1_4_13"]
BUCKET_CORRIDOR = "1_2_2"
BUCKET_BATHROOM = "1_3_11"
BUCKET_DOOR = "1_3_14"
BUCKET_BED = "1_4_13"

pir_buckets = {
        "1_2_2": "corridor",
        "1_3_11": "bathroom",
        "1_3_14": "door",
        "1_4_13": "bed",
    }

bucket_mapping = {
    "1_2_2": "corridor",
    "1_3_11": "bathroom",
    "1_3_14": "door",
    "1_4_13": "bed",
    "1_2_7": "battery_corridor",
    "1_3_10": "battery_bathroom",

}

BATTERY_BUCKETS = "1_2_7"

# ------------------------ INFLUXDB FUNCTIONS ------------------------

# Fetch data (inspired by the sif-viz-component fetch data structure)
def fetch_data(bucket, measurement, field):

    with InfluxDBClient(
        url=INFLUX_TOKEN, 
        org=INFLUX_ORG, 
        username=INFLUX_USER, 
        password=INFLUX_PASS, 
        verify_ssl=False) as client:
            p = {
                "_start": datetime(2024, 11, 28),  # Fetch data starting from January 1, 2023
                "_end": datetime(2024, 12, 3)   # Fetch data until December 31, 2023
            }

            query_api = client.query_api()
            tables = query_api.query(f'''
                                    from(bucket: "{bucket}") |> range(start: _start)
                                    |> filter(fn: (r) => r["_measurement"] == "{measurement}")
                                    |> filter(fn: (r) => r["_type"] == "{"sensor-value"}")
                                    |> filter(fn: (r) => r["_field"] == "{field}")
                                    ''', params=p)          
            obj = []
            
            base_logger.info(tables)
            for table in tables:
                for record in table.records:
                    val = {}
                    base_logger.info(record)
                    val["bucket"] = bucket
                    val["timestamp"] = record["_time"].timestamp() * 1000
                    val["value"] = record["_value"]
                    if bucket in BATTERY_BUCKETS:
                        val["field"] = record["_field"]
                        val["type"] = "battery"
                    else:
                        val["type"] = "sensor"
                    if len(val.keys()) != 0:
                        obj.append(val)

            return obj



In [102]:
def fetch_dataBattery_arrangement():
                batteryfetch = fetch_data("1_3_10", "battery", "soc")

                for record in batteryfetch:
                    record["_field"] = "1_2_7"  # BATTERY_BUCKETS

                df_battery = pd.DataFrame(batteryfetch)
                # df_battery["value"] = df_battery["_field"].map(bucket_mapping)
                # df_battery["timestamp"] = pd.to_datetime(df_battery["timestamp"], unit="ms")
                # df_battery = df_battery.sort_values("timestamp")

                print(df_battery)
                return batteryfetch


In [103]:
battery= fetch_dataBattery_arrangement()

      bucket     timestamp  value    type _field
0     1_3_10  1.730927e+12   63.0  sensor  1_2_7
1     1_3_10  1.730927e+12   98.0  sensor  1_2_7
2     1_3_10  1.730927e+12   92.0  sensor  1_2_7
3     1_3_10  1.730927e+12   99.0  sensor  1_2_7
4     1_3_10  1.730927e+12   99.0  sensor  1_2_7
...      ...           ...    ...     ...    ...
7859  1_3_10  1.733273e+12   95.0  sensor  1_2_7
7860  1_3_10  1.733274e+12   95.0  sensor  1_2_7
7861  1_3_10  1.733275e+12   95.0  sensor  1_2_7
7862  1_3_10  1.737031e+12   79.0  sensor  1_2_7
7863  1_3_10  1.737031e+12   99.0  sensor  1_2_7

[7864 rows x 5 columns]


In [104]:
from IPython.display import FileLink

# Convert the list to a DataFrame
battery_df = pd.DataFrame(battery)

# Save the DataFrame to a text file
battery_df.to_csv('data_sorted.txt', sep='\t', index=False)

# Provide a download link
FileLink('data_sorted.txt')

c:\Users\mario\Desktop\IoT_Cloud\sif-example-main\data_sorted.txt

In [55]:
def fetch_data_arrangement():   # Makes easier readable values for the model 

    # Fetch data for each bucket individually
    bathroomfetch = fetch_data(BUCKET_BATHROOM, "PIR", "roomID")
    corridorfetch = fetch_data(BUCKET_CORRIDOR, "PIR", "roomID")
    bedfetch = fetch_data(BUCKET_BED, "PIR", "roomID")
    doorfetch = fetch_data(BUCKET_DOOR, "door", "roomID2")

    # Add bucket identifier to each fetched dataset
    for record in bathroomfetch:
        record["bucket"] = "1_3_11"  # BUCKET_BATHROOM
    for record in corridorfetch:
        record["bucket"] = "1_2_2"   # BUCKET_CORRIDOR
    for record in bedfetch:
        record["bucket"] = "1_4_13"  # BUCKET_BED
    for record in doorfetch:
        record["bucket"] = "1_3_14"  # BUCKET_DOOR

    # Merge all fetched data
    all_fetched = bathroomfetch + corridorfetch + bedfetch + doorfetch

    # Create a pandas DataFrame
    df = pd.DataFrame(all_fetched)

    # Map bucket to value
    df["value"] = df["bucket"].map(bucket_mapping)

    # Convert timestamp and sort
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms")
    df = df.sort_values("timestamp")

    # Resulting dataframe
    print(df)
    
    return df

In [56]:
new_data=fetch_data_arrangement()

       bucket           timestamp     value    type
7626    1_2_2 2024-10-28 10:49:50  corridor  sensor
7627    1_2_2 2024-10-28 10:51:05  corridor  sensor
7628    1_2_2 2024-10-28 10:51:38  corridor  sensor
7629    1_2_2 2024-10-28 10:52:24  corridor  sensor
7630    1_2_2 2024-10-28 10:54:01  corridor  sensor
...       ...                 ...       ...     ...
17460  1_3_14 2024-11-25 02:39:31      door  sensor
7624   1_3_11 2024-11-25 02:39:42  bathroom  sensor
17461  1_3_14 2024-11-25 02:39:42      door  sensor
7625   1_3_11 2024-11-25 02:39:50  bathroom  sensor
17462  1_3_14 2024-11-25 02:39:50      door  sensor

[17463 rows x 4 columns]


In [32]:
from IPython.display import FileLink

# Save the DataFrame to a text file
new_data.to_csv('data_sorted.txt', sep='\t', index=False)

# Provide a download link
FileLink('data_sorted.txt')

c:\Users\mario\Desktop\IoT_Cloud\sif-example-main\data_sorted.txt

In [33]:
def data_sorting(df):   

    df_sorted = df.sort_values(by='timestamp')
    df_sorted['human_readable_time'] = pd.to_datetime(df_sorted['timestamp'], unit='ms')
    print(df_sorted.head(40))

    return df_sorted


In [34]:
data_sorted=data_sorting(new_data)

   bucket           timestamp     value    type human_readable_time
1   1_2_2 2024-11-24 18:55:18  corridor  sensor 2024-11-24 18:55:18
2  1_4_13 2024-11-24 18:57:36       bed  sensor 2024-11-24 18:57:36
0  1_3_11 2024-11-25 02:39:50  bathroom  sensor 2024-11-25 02:39:50
3  1_3_14 2024-11-25 02:39:50      door  sensor 2024-11-25 02:39:50


In [17]:
#withouy seasonality 
def calculate_stays(df):
    stays = []
    current_stay = None

    for index, row in df.iterrows():
        if current_stay is None:
            current_stay = {
                'room': row['value'],
                'start': row['timestamp'],
                'end': row['timestamp']
            }
        elif row['value'] == current_stay['room']:
            current_stay['end'] = row['timestamp']
        else:
            current_stay['duration'] = (current_stay['end'] - current_stay['start']).total_seconds()
            stays.append(current_stay)
            current_stay = {
                'room': row['value'],
                'start': row['timestamp'],
                'end': row['timestamp']
            }

    if current_stay is not None:
        current_stay['duration'] = (current_stay['end'] - current_stay['start']).total_seconds()
        stays.append(current_stay)

    stays_df = pd.DataFrame(stays)
    print(stays_df)
    return stays_df

stays_df = calculate_stays(data_sorted)

           room               start                 end  duration
0      corridor 2024-10-28 10:49:50 2024-11-06 20:53:37  813827.0
1      bathroom 2024-11-06 20:57:17 2024-11-06 20:57:33      16.0
2      corridor 2024-11-06 20:57:33 2024-11-06 20:57:33       0.0
3      bathroom 2024-11-06 20:57:53 2024-11-06 20:58:14      21.0
4      corridor 2024-11-06 20:58:14 2024-11-06 21:14:15     961.0
...         ...                 ...                 ...       ...
11661      door 2024-11-25 02:39:31 2024-11-25 02:39:31       0.0
11662  bathroom 2024-11-25 02:39:42 2024-11-25 02:39:42       0.0
11663      door 2024-11-25 02:39:42 2024-11-25 02:39:42       0.0
11664  bathroom 2024-11-25 02:39:50 2024-11-25 02:39:50       0.0
11665      door 2024-11-25 02:39:50 2024-11-25 02:39:50       0.0

[11666 rows x 4 columns]


In [18]:
import numpy as np

def calculate_seasonal_stays(df):
    # Extract day of the week and hour from the start time
    df['day_of_week'] = df['start'].dt.dayofweek
    df['hour_of_day'] = df['start'].dt.hour

    # Group by room, day of the week, and hour of the day
    grouped = df.groupby(['room', 'day_of_week', 'hour_of_day'])

    # Calculate the number of visits and average duration
    seasonal_stats = grouped['duration'].agg(['count', 'mean']).reset_index()
    seasonal_stats.rename(columns={'count': 'number_of_visits', 'mean': 'average_duration'}, inplace=True)

    # Sort by day of the week and hour of the day
    seasonal_stats = seasonal_stats.sort_values(by=['day_of_week', 'hour_of_day'])

    print(seasonal_stats)
    return seasonal_stats

seasonal_stays_df = calculate_seasonal_stays(stays_df)

         room  day_of_week  hour_of_day  number_of_visits  average_duration
0    bathroom            0            0                15        167.200000
166      door            0            0                14          2.642857
1    bathroom            0            1                 2       1823.500000
167      door            0            1                 2          3.000000
2    bathroom            0            2                 9          1.666667
..        ...          ...          ...               ...               ...
271      door            6           21                40         10.650000
108  bathroom            6           22                32         20.312500
272      door            6           22                32         65.531250
109  bathroom            6           23                40          8.975000
273      door            6           23                40         13.300000

[274 rows x 5 columns]


In [19]:
from IPython.display import FileLink

# Save the seasonal_stays_df DataFrame to a text file
seasonal_stays_df.to_csv('seasonal_stays.txt', sep='\t', index=False)

# Provide a download link
FileLink('seasonal_stays.txt')

c:\Users\mario\Desktop\IoT_Cloud\sif-example-main\seasonal_stays.txt

In [20]:
def identify_outliers(df, seasonal_stats):
    # Merge the original dataframe with the seasonal statistics
    merged_df = df.merge(seasonal_stats, on=['room', 'day_of_week', 'hour_of_day'], how='left')

    # Identify outliers where the duration is longer than the average duration
    merged_df['is_outlier'] = merged_df['duration'] > merged_df['average_duration']

    # Filter out the outliers
    outliers_df = merged_df[merged_df['is_outlier']]

    print(outliers_df)
    return outliers_df

outliers_df = identify_outliers(stays_df, seasonal_stays_df)

           room               start                 end  duration  \
3      bathroom 2024-11-06 20:57:53 2024-11-06 20:58:14      21.0   
4      corridor 2024-11-06 20:58:14 2024-11-06 21:14:15     961.0   
5           bed 2024-11-06 21:17:52 2024-11-06 21:19:21      89.0   
9           bed 2024-11-06 21:23:26 2024-11-06 21:24:49      83.0   
11          bed 2024-11-06 21:25:10 2024-11-06 21:26:04      54.0   
...         ...                 ...                 ...       ...   
11646  bathroom 2024-11-25 01:39:36 2024-11-25 02:34:39    3303.0   
11648  bathroom 2024-11-25 02:34:47 2024-11-25 02:34:54       7.0   
11651      door 2024-11-25 02:35:03 2024-11-25 02:38:55     232.0   
11654  bathroom 2024-11-25 02:39:01 2024-11-25 02:39:05       4.0   
11658  bathroom 2024-11-25 02:39:14 2024-11-25 02:39:18       4.0   

       day_of_week  hour_of_day  number_of_visits  average_duration  \
3                2           20                 2         18.500000   
4                2           

In [21]:

async def CreateOccupancyModelFunction():
   #base_logger.info("Function ocupancy_model_creation called.")

    # base_logger.info("Function create occupancy model called.")
    # data_recieved = await request.json()
    # base_logger.info(f"Function create_occupancy_model_function received data: {data_recieved}")

    #base_logger.info("Fetching data")
    new_data=fetch_data_arrangement()
    data_sorted=data_sorting(new_data)

   # base_logger.info("Calculating average times")
    stays_df = calculate_stays(data_sorted)  
    seasonal_stays_df = calculate_seasonal_stays(stays_df)

    #base_logger.info("Calculating outliers")
    outliers_df = identify_outliers(stays_df, seasonal_stays_df)

    # base_logger.info("Saving ocupancy model to minio")
    # save_model_minio(time_stats)  # cambia time_stats por tu output del modelo final 
    # load_latest_model_minio()   
    # base_logger.info("model trained and stored")

    return {"status": 200, "message": "Model trained and stored"}

In [22]:
intento = await CreateOccupancyModelFunction()


       bucket           timestamp     value
7626    1_2_2 2024-10-28 10:49:50  corridor
7627    1_2_2 2024-10-28 10:51:05  corridor
7628    1_2_2 2024-10-28 10:51:38  corridor
7629    1_2_2 2024-10-28 10:52:24  corridor
7630    1_2_2 2024-10-28 10:54:01  corridor
...       ...                 ...       ...
17460  1_3_14 2024-11-25 02:39:31      door
7624   1_3_11 2024-11-25 02:39:42  bathroom
17461  1_3_14 2024-11-25 02:39:42      door
7625   1_3_11 2024-11-25 02:39:50  bathroom
17462  1_3_14 2024-11-25 02:39:50      door

[17463 rows x 3 columns]
     bucket           timestamp     value human_readable_time
7626  1_2_2 2024-10-28 10:49:50  corridor 2024-10-28 10:49:50
7627  1_2_2 2024-10-28 10:51:05  corridor 2024-10-28 10:51:05
7628  1_2_2 2024-10-28 10:51:38  corridor 2024-10-28 10:51:38
7629  1_2_2 2024-10-28 10:52:24  corridor 2024-10-28 10:52:24
7630  1_2_2 2024-10-28 10:54:01  corridor 2024-10-28 10:54:01
7631  1_2_2 2024-10-28 10:54:19  corridor 2024-10-28 10:54:19
7632  1_2_2 

C:\Users\mario\AppData\Local\Temp\ipykernel_18048\1756240550.py:1: RuntimeWarning: coroutine 'CreateOccupancyModelFunction' was never awaited
  intento = await CreateOccupancyModelFunction()
